## Train Model With One Layer on Top of CamemBERT. REGULARIZATION.

In [1]:
import os
import numpy as np

from silence_tensorflow import silence_tensorflow
silence_tensorflow()  # silence TF warnings
import tensorflow as tf

from dataProcessing import load_file, encode_data, insert_target, processingScriber00
from transformers import AutoTokenizer
from transformers import TFCamembertForMaskedLM
from datetime import datetime
import json
import sys

In [2]:
### instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained("jplu/tf-camembert-base", do_lower_case=True)

### punctuation encoder
punctuation_enc = {
    'O': 0,
    'PERIOD': 1,
}

### hyper-parameters
vocabSize = 32005
sequenceSize = 32
batchSize = 32
learat = 1e-5
trainLayInd = 0
training = True
hidden_dropout_prob = 0.5
numEpo = 10

listHyper0 = ['sequenceSize', 'batchSize', 'learat', 'trainLayInd',
              'training', 'hidden_dropout_prob', 'numEpo']
listHyper1 = [str(sequenceSize), str(batchSize), str(learat), str(trainLayInd),
              str(training), str(hidden_dropout_prob), str(numEpo)]
time = datetime.now().strftime("%Y%m%d_%H%M%S")
save_path = 'ModelsExpScriber/{}/'.format(time)
os.mkdir(save_path)

In [3]:
### Training Dataset
print('\nProcessing Data ... ')

# name of dataset with sentences
data_name = "Scriber"

# file name
trainDataName = 'Data' + data_name + '/' + 'raw.processed.Train_01.txt'

# from sentences to list of words+punctuation
data_train = load_file(processingScriber00(trainDataName))

# encode data and insert target
X_train_, y_train_ = encode_data(data_train, tokenizer, punctuation_enc)
X_train = insert_target(X_train_, sequenceSize)
y_train = np.asarray(y_train_)

# ### get only a fraction of data 
# n = 96
# X_train = X_train[0:n]
# y_train = y_train[0:n]

# build the datasets
trainDataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=500000).batch(batchSize)

print("\nTraining Dataset Tensor Shape = ", X_train.shape)


Processing Data ... 

Training Dataset Tensor Shape =  (96, 32)


## Build the Model

In [4]:
### get configuration file
modelBERT = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")
configBERT = modelBERT.config

In [5]:
### change dropout probability
configBERT.hidden_dropout_prob = hidden_dropout_prob

In [6]:
print('\nBulding the Regularized Model ... ')

bert_input = tf.keras.Input(shape=(sequenceSize), dtype='int32', name='bert_input')
x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base", config=configBERT)(bert_input, training=training)[0]
x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
dense_out = tf.keras.layers.Dense(2)(x)

model = tf.keras.Model(bert_input, dense_out, name='model')

# define the loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def loss(model, x, y):
    y_ = model(x)
    return loss_object(y_true=y, y_pred=y_)

# func to calculate the gradients
def grad(model, inputs, targets, trainLayInd):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return loss_value, tape.gradient(loss_value, model.trainable_variables[trainLayInd:])

# define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learat)


Bulding the Regularized Model ... 


## Train the Model

In [7]:
print("\nEperiment Folder: ", time)
print("\nHyperparameters:")
print('sequenceSize = ', sequenceSize)
print('batchSize = ', batchSize)
print('leaRat = ', learat)
print('train Layer Index = ', trainLayInd)
print('Hidden Dropout Probability = ', hidden_dropout_prob)
print('numEpo = ', numEpo)

epoch_loss_avg = tf.keras.metrics.Mean()
epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

train_loss_results = []
train_accuracy_results = []

checkpoint_path = save_path + "cp-{epoch:03d}.ckpt"

print("\nTraining the Model ... ")

tmpTrain = np.inf
for epoch in range(1, numEpo+1):

    # training loop
    for x, y in trainDataset:
        # optimize the model
        loss_value, grads = grad(model, x, y, trainLayInd)
        optimizer.apply_gradients(zip(grads, model.trainable_variables[trainLayInd:]))

        # track progress
        epoch_loss_avg.update_state(loss_value)
        epoch_accuracy.update_state(y, model(x))

    # end epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    print("\nEpoch {:03d}: (Training)   Loss: {:.3f}, Accuracy: {:.3%}".format(epoch, epoch_loss_avg.result(), epoch_accuracy.result()))

    # # save model if new min for train loss is found
    # if epoch_loss_avg.result().numpy() < tmpTrain:
    tmpTrain = epoch_loss_avg.result().numpy()
    model.save_weights(checkpoint_path.format(epoch=epoch))

    epoch_loss_avg.reset_states()
    epoch_accuracy.reset_states()


Training the Model ... 

Eperiment Folder:  20200619_235839

Hyperparameters:
sequenceSize:  32
batchSize:  32
leaRat:  1e-05
train Layer Index:  0
Hidden Dropout Probability:  0.5
numEpo:  3

Epoch 001: (Training)   Loss: 13.601, Accuracy: 90.625%

Epoch 002: (Training)   Loss: 12.706, Accuracy: 90.625%

Epoch 003: (Training)   Loss: 12.900, Accuracy: 90.625%


## Output Training Details On Log File

In [8]:
nameLogFile = 'log.txt'
logFile = open(save_path + nameLogFile, "w")

# write name of model
logFile.write("\n" + time + "\n\n")

# write hyper parameters
for i in range(len(listHyper0)):
    logFile.write(listHyper0[i] + ":  " + listHyper1[i] + "\n")

# write training details
logFile.write('\nTRAINING')
trainLossArr = np.asarray(train_loss_results)
trainAccArr = np.asarray(train_accuracy_results)
for i in range(numEpo):
    epoch = i+1
    logFile.write("\nEpoch {:03d}:   Loss: {:7.4f},   Accuracy: {:7.4%}".format(epoch, trainLossArr[i], trainAccArr[i]))

## Evaluate the model, write the details on the logFile

In [9]:
### get the dataset

# name of dataset with sentences
data_name = "Scriber"
fileName = 'Data' + data_name + '/' + 'raw.processed.Test_01.txt'

# from sentences to list of words+punctuation
data = load_file(processingScriber00(fileName))

# encode and insert target
X_, y_ = encode_data(data, tokenizer, punctuation_enc)
X = insert_target(X_, sequenceSize)
y = np.asarray(y_)

# ### get only an n of the data.
# n = 64
# print(X.shape)
# X = X[0:n]
# y = y[0:n]
# print(X.shape)

# one hot encode the labels
y = tf.one_hot(y, len(punctuation_enc), dtype='int64').numpy()

dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batchSize)
print("\nTest Dataset Tensor Shape = ", X.shape)

(21009, 32)
(64, 32)

Test Dataset Tensor Shape =  (64, 32)


In [10]:
### build and compile model

bert_input = tf.keras.Input(shape=(sequenceSize), dtype='int32', name='bert_input')
x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base", config=configBERT)(bert_input, training=False)[0]
x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
dense_out = tf.keras.layers.Dense(len(punctuation_enc), activation='softmax')(x)

model = tf.keras.Model(bert_input, dense_out, name='model')

model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(from_logits=False),
              metrics=[tf.keras.metrics.Recall(class_id=0, name='Rec_0'),
                       tf.keras.metrics.Precision(class_id=0, name='Prec_0'),
                       tf.keras.metrics.Recall(class_id=1, name='Rec_1'),
                       tf.keras.metrics.Precision(class_id=1, name='Prec_1'),
                      ])

In [11]:
### Get List of the Models in the Output Folder
modelsLst = []
for r, d, f in os.walk(save_path):
    for file in sorted(f):
        if ".index" in file:
            modelsLst.append(file[:-6])

In [12]:
# compute f1 score
def compF1(rec, pre):
    if pre + rec == .0:
        return .0
    else:
        return 2 * (pre*rec) / (pre+rec)

In [13]:
### Evaluate the Models

print("\nEvaluate Models")

logFile.write('\n\nEVALUATION\n')
for i in range(len(modelsLst)):
    checkpointPath = save_path + modelsLst[i]
    print(checkpointPath)

    # load weights
    model.load_weights(checkpointPath)

    # evaluate
    evaluation = model.evaluate(dataset)
    
    f1_0 = compF1(evaluation[1],evaluation[2])
    f1_1 = compF1(evaluation[3],evaluation[4])
    print("F1_0 = {:10.7f} - F1_1 = {:10.7f}".format(f1_0, f1_1))
    
    # write details on log files
    logFile.write(modelsLst[i])
    logFile.write(" - Loss = {:7.4f} - Rec_0 = {:6.4f} - Pre_0 = {:6.4f} - F1_0 = {:10.7f} - Rec_1 = {:6.4f} - Pre_1 = {:6.4f} - F1_1 = {:10.7f}\n".format(evaluation[0], evaluation[1], evaluation[2], f1_0, evaluation[3], evaluation[4], f1_1))

logFile.close()


Evaluate Models
ModelsExpScriber/20200619_235839/cp-001.ckpt
2/2 [==============================] - 3s 1s/step - loss: 9.0527 - Rec_0: 1.0000 - Prec_0: 0.9531 - Rec_1: 0.0000e+00 - Prec_1: 0.0000e+00
F1_0 =  0.9760000 - F1_1 =  0.0000000
ModelsExpScriber/20200619_235839/cp-002.ckpt
2/2 [==============================] - 3s 1s/step - loss: 10.7712 - Rec_0: 1.0000 - Prec_0: 0.9531 - Rec_1: 0.0000e+00 - Prec_1: 0.0000e+00
F1_0 =  0.9760000 - F1_1 =  0.0000000
ModelsExpScriber/20200619_235839/cp-003.ckpt
2/2 [==============================] - 3s 1s/step - loss: 10.1998 - Rec_0: 1.0000 - Prec_0: 0.9531 - Rec_1: 0.0000e+00 - Prec_1: 0.0000e+00
F1_0 =  0.9760000 - F1_1 =  0.0000000
